# Data-preparation for Prediction of "Sachkonto" - from Table SAP BSAK

# Imports

In [13]:
import pandas as pd
import numpy as np
from utils_bsak import printSamplesFromSaktos
from utils_bsak import is_date_column, is_decimal_column, convert_column_decimal2float

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from utils_bsak import target_min_value_records

import joblib

import xgboost as xgb


# Load Data & First Overview

#### Names and Paths for Loading Raw-Data

In [ ]:
# Files for Data-Loading

folder_raw_data = "../data_raw"

# I have three files containing data:
file_csv1 = r"\Export_bsak_0124-prctr.csv".replace("\\", "/")
file_csv2 = r"\Export_bsak_0224-prctr.csv".replace("\\", "/")
file_csv3 = r"\export_bsak_010123-311223-SaktoExclude.csv".replace("\\", "/")

files_csv = [file_csv1, file_csv2, file_csv3]




#### Names and Paths for Writting Preprocessed Data

In [15]:
# Files for Preprocessed Data

model_name = "Sachkonto_stratified" # name of the model that is going to be used

folder_preprocessed_data = "../data_preprocessed/"
file_joblib_data = 'Data_' + model_name + '.pkl'
path_joblib_data = folder_preprocessed_data + file_joblib_data # path for dumping preprocessed data
folder_web_mappings = '../demo_web_page/mappings' # folder for encoder-mapping etc. to be used by web-page


### Load Data

In [ ]:
df = pd.DataFrame()
for _file in files_csv:
    path_data = folder_raw_data + _file
    df_new = pd.read_csv(path_data, sep=';', encoding='latin1')
    df = pd.concat([df, df_new], ignore_index=True)

# Erste Zeilen anzeigen, um sicherzustellen, dass die Daten korrekt geladen wurden
print(df.head(2))

In [ ]:
df.shape

## Percentage Distribution of Target-Values: "Sachkonto"

In [ ]:
print("Percentage of data for a specific 'Sachkonto'")
print(df["Sachkonto"].value_counts(normalize=True) * 100)
print("--------------------------------")
print("Count of data for a specific 'Sachkonto'")
print(df["Sachkonto"].value_counts())
print("--------------------------------")

# Data Cleaning

In [ ]:
target = 'Sachkonto'
proceed = target in df.columns
#proceed = False

if proceed:
    # drop empty, i.e. nan-only rows:
    df = df.dropna(axis=0, how='all')


    # drop empty, i.e. nan-only columns:
    df = df.dropna(axis=1, how='all')


    # drop duplicate rows:
    df = df.drop_duplicates()


    # drop rows with target being nan:
    df = df.dropna(subset=[target], axis=0)
    

    # keep only rows with target not being empty string:
    df = df[df[target] != ""]
    

    # keep only columns with non-constant value:
    df = df.loc[:, df.nunique() > 1]


    # remove columns containing dates - no time-series analysis components in this notebook:
    # define a function to identify date columns:
    import re

    date_cols = [col for col in df.columns if is_date_column(df[col])]
    df = df.drop(columns=date_cols)

    # Identify decimal columns:
    decimal_cols = [col for col in df.columns if is_decimal_column(df[col])]

    # Convert decimal-colums to floats:
    for col in decimal_cols:
        df[col] = convert_column_decimal2float(df[col])

    # Label-encode all columns of type "object":
    from sklearn.preprocessing import LabelEncoder

    columns_to_encode = df.dtypes[df.dtypes == "object"].index.to_list()
    column_encoders = {}
    label_encoder = LabelEncoder()

    for col in columns_to_encode:
        label_encoder = LabelEncoder()
        df[col] = label_encoder.fit_transform(df[col].astype(str))
        # save encoding for each column
        inverse_dict = { k: i for i,k in enumerate(label_encoder.classes_)}
        column_encoders[col] = inverse_dict


    # drop columns that contain only identifiers (that we do not want to analyze further here):
    identifier_columns = ["Referenz", "Ausgleichsbeleg", "Zuordnung", "Belegnummer", "Auftrag", "Einkaufsbeleg", "Rechnungsbezug"]
    
    for id_col in identifier_columns:
        if id_col in df.columns:
            df = df.drop(id_col, axis='columns')
   
    # finally drop columns that contain data that we can not / do not want to analyze - drop this columns:
    # 'Text': to evaluate this would require a model on it's own - not (yet) justified
    # 'Zuordnung': possible target-leakage?
    # 'Hauptbuchkonto' : possible target-leakage? won't be available, when process is in frontend anyway.

    unwanted_columns = ["Text", "Zuordnung", "Hauptbuchkonto"]
    nan_columns = list(df.columns[df.isna().sum() >0].values)
    print(f"nan_columns: {nan_columns}")
    unwanted_columns.extend(nan_columns)
    for u_col in unwanted_columns:
        if u_col in df.columns:
            df.drop(columns=[u_col], inplace=True)

print("Done!")


# Train-Test Split & Feature Selection with Boruta

## Stratified Train-Test Split:

In [ ]:
# stratified split of data

# for stratification, all target classes have to have more than 1 record - we choose 4 as minimum here:
stratifiable_target_values = target_min_value_records(dataframe=df, target_column=target, min_value_records=4).astype(int)

print("--------------------------------")
print(f"df.shape with all targets: {df.shape}")
df = df[df[target].isin(stratifiable_target_values)]
print(f"df.shape with stratifiable targets: {df.shape}")
print("--------------------------------")

# prepare target encoding:
target_label_encoder = LabelEncoder()
y = df[target]
target_label_encoder.fit(y)

df_train, df_test = train_test_split(df, test_size=0.2, random_state=42, stratify=y)

print("--------------------------------")
print(f"df_train.shape : {df_train.shape}")
print(f"df_test.shape : {df_test.shape}")
print("--------------------------------")

# split target from data:
X_train = df_train.drop(target, axis=1)
X_test = df_test.drop(target, axis=1)
y_train = target_label_encoder.transform(df_train[target].values)
y_test = target_label_encoder.transform(df_test[target].values)

print("--------------------------------")
print("y_train unique values:")
print(np.sort(np.unique(y_train)))
print("y_test unique values:")
print(np.sort(np.unique(y_test)))
print("--------------------------------")


In [ ]:
# info needed for webpage script.js:

Steuerkennzeichen_dict = column_encoders['Steuerkennzeichen']
Steuerkennzeichen_dict

In [ ]:
# info needed for webpage script.js:

target_dict = { i: k for i,k in enumerate(target_label_encoder.classes_)}
target_dict

# Store the Transformed and Encoded Data and Labels:

In [23]:
# Store data and LabelEncoder for later use:
import joblib


In [ ]:
# Store data and LabelEncoder for later use:

data = {
    'X_train': X_train,
    'X_test': X_test,
    'y_train': y_train,
    'y_test': y_test,
    'column_encoders' : column_encoders,
    'target_dict': target_dict,
    'Steuerkennzeichen_dict': Steuerkennzeichen_dict
}

joblib.dump(data, path_joblib_data)


## Export some Data to JSON for the Web-Application

In [25]:
import joblib
import json
import os

class NumpyEncoder(json.JSONEncoder):
    """
    Transforms NumPy-types to JSON compatible Python types.
    """
    def default(self, obj):
        if isinstance(obj, (np.integer, np.int64)):
            return int(obj)
        elif isinstance(obj, (np.floating, np.float32, np.float64)):
            return float(obj)
        elif isinstance(obj, (np.ndarray,)):
            return obj.tolist()
        return super().default(obj)

def export_mapping_for_web(joblib_file_path, output_folder, keys_to_export):
    """
    Loads a joblib file, extracts certain keys and exports them to single JSON-files.
    
    :param joblib_file_path: path to a .joblib-file containing the data
    :param output_folder: folder to write the JSON-files to
    :param keys_to_export: list of keys (e.g. ['target_dict', 'Steuerkennzeichen_dict'])
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    data = joblib.load(joblib_file_path)

    for key in keys_to_export:
        if key in data:
            output_path = os.path.join(output_folder, f"{key}.json")
            with open(output_path, "w", encoding="utf-8") as f:
                json.dump(data[key], f, ensure_ascii=False, indent=2, cls=NumpyEncoder)
                print(f"{key} exported to {output_path}")
        else:
            print(f"Key '{key}' not found in data.")


In [ ]:
# do the export:

export_keys = ["target_dict", "Steuerkennzeichen_dict"] # what to export into single json files named like this keys

export_mapping_for_web(path_joblib_data, folder_web_mappings, export_keys)
